<a href="https://colab.research.google.com/github/hivani/-Question-Answering-system-using-an-LLM-/blob/main/steps_AI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Advanced Chunking of Data***

In [2]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import json
from sentence_transformers import SentenceTransformer, util

# Load JSON Data
with open('/content/scraped_data.json') as f:
    data = json.load(f)

# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to chunk data based on semantic similarity
def chunk_data(doc):
    sentences = doc.split('. ')
    embeddings = model.encode(sentences)
    clusters = util.community_detection(embeddings, min_community_size=2)
    chunks = [' '.join([sentences[idx] for idx in cluster]) for cluster in clusters]
    return chunks

# Chunk all documents
all_chunks = []
for entry in data:
    chunks = chunk_data(entry['content'])
    for chunk in chunks:
        all_chunks.append({
            'text': chunk,
            'meta': {'web_link': entry.get('web_link', 'N/A')}
        })

print(f"Total chunks created: {len(all_chunks)}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total chunks created: 5


# ***Convert Data to Embedding Vectors***

In [4]:
# Convert text chunks to embedding vectors
chunk_texts = [chunk['text'] for chunk in all_chunks]
chunk_embeddings = model.encode(chunk_texts)


In [5]:
pip install grpcio==1.63.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1


In [6]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.9 MB/s eta 0:00:00


## **Create Vector Database **

In [7]:
import faiss
import numpy as np

# Convert the embeddings to float32
chunk_embeddings = np.array(chunk_embeddings).astype('float32')

# Create a FAISS index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexHNSWFlat(dimension, 32)  # HNSW index with 32 neighbors
index.hnsw.efConstruction = 200

# Add embeddings to the index
index.add(chunk_embeddings)

# Store metadata (in-memory for simplicity, you may use a database or file)
metadata = [{'web_link': chunk['meta']['web_link']} for chunk in all_chunks]


# ** Retrieval and Re-ranking**

In [8]:
# Use a simple query expansion technique (e.g., synonyms)
def expand_query(query):
    synonyms = ["synonym1", "synonym2"]  # Example synonyms
    expanded_query = f"{query} {' '.join(synonyms)}"
    return expanded_query

# Expand the query
query = "example query"
expanded_query = expand_query(query)


In [9]:
from transformers import pipeline

# Use a simple query expansion technique (e.g., synonyms)
def expand_query(query):
    synonyms = ["synonym1", "synonym2"]  # Example synonyms
    expanded_query = f"{query} {' '.join(synonyms)}"
    return expanded_query

# Expand the query
query = "example query"
expanded_query = expand_query(query)


# **Hybrid Retrieval Methods**

In [10]:

# Encode the query
query_embedding = model.encode([expanded_query]).astype('float32')

# Search the index
k = 10  # Number of nearest neighbors
distances, indices = index.search(query_embedding, k)

# Extract results
retrieved_chunks = [chunk_texts[i] for i in indices[0]]
retrieved_metadata = [metadata[i] for i in indices[0]]

print(f"Retrieved Chunks: {retrieved_chunks}")
print(f"Retrieved Metadata: {retrieved_metadata}")


Retrieved Chunks: ['This guide summarizes the ways that applications can be fine-tuned to gain additional speedups by leveraging Hopper GPU Architectureâ\x80\x99s features.</p>\r\n</dd>\r\n<dt><a class="reference internal" href="ada-tuning-guide/index.html"><span class="doc">Ada Tuning Guide</span></a></dt><dd><p>The NVIDIAÂ® Ada GPU architecture is NVIDIAâ\x80\x99s latest architecture for CUDAÂ® compute applications This guide summarizes the ways that applications can be fine-tuned to gain additional speedups by leveraging NVIDIA Ampere GPU Architectureâ\x80\x99s features.</p>\r\n</dd>\r\n<dt><a class="reference internal" href="hopper-tuning-guide/index.html"><span class="doc">Hopper Tuning Guide</span></a></dt><dd><p>Hopper GPU Architecture is NVIDIAâ\x80\x99s 9th-generation architecture for CUDA compute applications This guide summarizes the ways that applications can be fine-tuned to gain additional speedups by leveraging Turing architectural features.</p>\r\n</dd>\r\n<dt><a class=

In [11]:
# Re-rank the retrieved results
reranked_results = util.semantic_search(model.encode(query), model.encode(retrieved_chunks), top_k=10)

# Flatten the re-ranked results
reranked_chunks = [retrieved_chunks[idx['corpus_id']] for idx in reranked_results[0]]


In [12]:
from transformers import pipeline

# Initialize QA pipeline
qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2')


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [13]:
# Use the top re-ranked result for question answering
context = reranked_chunks[0]
answer = qa_pipeline({'question': query, 'context': context})

print(f"Answer: {answer['answer']}")


Answer: NVIDIA Ampere GPU Architecture


In [14]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [16]:
! pip install streamlit -q

In [17]:
!wget -q -O - ipv4.icanhazip.com

34.125.202.28


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.202.28:8501

npx: installed 22 in 3.926s
your url is: https://dark-rules-move.loca.lt
2024-07-16 15:06:39.550 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 589, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 8, in <module>
    expanded_query = expand_query(query)
NameError: name 'expand_query' is not defined
